# Chatbot IA para Vendas de Carros
## Sistema Integrado Telegram + RAG + LLM Local (OLLAMA)

Este notebook implementa um chatbot inteligente para vendas de carros usando Telegram como interface, RAG (Retrieval Augmented Generation) para contexto e LLM local para processamento de linguagem natural.


### 1. Configuração do Ambiente

Primeiro, precisamos instalar todas as dependências necessárias para o projeto.



In [ ]:
# Instalação das bibliotecas necessárias
!pip install pyTelegramBotAPI
!pip install llama-index
!pip install python-telegram-bot --upgrade

### 2. Importações e Configurações Iniciais

Importamos todas as bibliotecas necessárias para o projeto.

In [ ]:
import telebot
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from llama_index.core.prompts import ChatPromptTemplate, PromptTemplate

### 3. Configuração do Bot Telegram

Configure seu bot Telegram usando o token obtido do BotFather.

In [ ]:
# Configuração do token do bot
bot_key = 'SEU_TOKEN_AQUI'
bot = telebot.TeleBot(bot_key, parse_mode=None)

### 4. Configuração do Sistema LLM e Embeddings

Configuramos o modelo LLM local e o sistema de embeddings.

In [ ]:
# Configuração do LLM e embeddings
Settings.llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    base_url="http://localhost:11434"
)

### 5. Carregamento e Indexação dos Dados

Carregamos os documentos e criamos o índice vetorial.

In [ ]:
# Carregamento dos documentos
data = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(data)

### 6. Configuração dos Prompts

Definimos os prompts que serão usados pelo sistema.

In [ ]:
# Prompt principal para o assistente de vendas
qa_prompt_str = (
    "As informações de contexto estão abaixo.\n"
    "Seu nome é uma Assistente de vendas.\n"
    "Está oferecendo um carro usado e não novo. \n"
    "Seja simpático.\n"
    "Responda sempre em português.\n"
    "Haja como se você estivesse vendendo o Siena 1.4, ano 2010, baseado na base de dados fornecida.\n"
    "No final de cada resposta, lembre o cliente que para negociações reais, utilize o comando /help para obter o contato do vendedor.\n"
    "{context_str}\n"
    "---------------------\n"
    "Dadas as informações de contexto, responda à pergunta de forma breve: {query_str}\n"
)

qa_prompt = PromptTemplate(qa_prompt_str)

In [ ]:
# Template de refinamento
refine_template_str = (
    "responda a {query_str} com uma resposta breve"
    " Seja fiel a sua base de dados."
    " NÃO INVENTE VALORES OU PROPOSTA"
    " NÃO FECHE NEGOCIO DIRECIONE AO /help PARA FECHAR E MARCAR VISITA"
)
refine_template = PromptTemplate(refine_template_str)

### 7. Configuração do Motor de Consulta

Configuramos o motor que processará as consultas dos usuários.

In [ ]:
# Configuração do query engine
query_engine = index.as_query_engine(
    text_qa_template=qa_prompt,
    refine_template=refine_template
)

### 8. Implementação dos Handlers do Bot

Implementamos os handlers que processarão as mensagens do Telegram.

In [ ]:
# Handler de boas-vindas
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, """
    Olá! Sou um bot que pode responder perguntas baseadas nos documentos carregados.
    Digite sua pergunta e farei o possível para respondê-la.
    """)

# Handler principal para processamento de mensagens
@bot.message_handler(func=lambda message: True)
def handle_query(message):
    try:
        # Mensagem de processamento
        processing_msg = bot.reply_to(message, "Processando sua pergunta...")
        
        # Obtém resposta do LLM
        response = query_engine.query(message.text)
        
        # Envia resposta
        bot.edit_message_text(
            chat_id=message.chat.id,
            message_id=processing_msg.message_id,
            text=str(response)
        )
    except Exception as e:
        bot.edit_message_text(
            chat_id=message.chat.id,
            message_id=processing_msg.message_id,
            text=f"Desculpe, ocorreu um erro: {str(e)}"
        )

### 9. Iniciando o Bot

Finalmente, iniciamos o bot para começar a processar mensagens.

In [ ]:
if __name__ == "__main__":
    print("Bot iniciado...")
    bot.polling()

### Estrutura de Diretórios

Para que o projeto funcione corretamente, mantenha a seguinte estrutura:

```
projeto/
├── data/
│   └── [seus arquivos de dados]
├── notebook.ipynb
└── README.md
```

### Notas Importantes

1. **Segurança**:
   - Nunca compartilhe seu token do bot
   - Mantenha seus dados sensíveis em variáveis de ambiente
   - Faça backup regular da base de dados

2. **Manutenção**:
   - Atualize regularmente as bibliotecas
   - Monitore o uso de recursos
   - Faça backup dos logs importantes

3. **Customização**:
   - Ajuste os prompts conforme necessário
   - Adicione novos comandos conforme a demanda
   - Mantenha a base de dados atualizada

### Troubleshooting

Problemas comuns e soluções:

1. **Erro de conexão com Ollama**:
   - Verifique se o serviço está rodando
   - Confirme a URL base
   - Verifique as portas

2. **Erro no Telegram**:
   - Verifique o token
   - Confirme as permissões do bot
   - Verifique a conexão com a API

3. **Problemas com RAG**:
   - Verifique a estrutura dos documentos
   - Confirme o formato dos dados
   - Verifique os logs de indexação

### Próximos Passos

Abaixo um codigo completo em python de uma possivel implementação.

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from llama_index.core.prompts import ChatPromptTemplate, PromptTemplate
import telebot
from telebot import types
from telebot.handler_backends import State, StatesGroup
from telebot.storage import StateMemoryStorage
import json
import os
from datetime import datetime

class EnhancedTelegramBot:
    def __init__(self, bot_token, data_dir="./data"):
        self.bot = telebot.TeleBot(bot_token, parse_mode='HTML')
        self.data_dir = data_dir
        self.conversation_history = {}
        self.photo_requests = set()
        self.whatsapp_number = "aqui"  # Substitua aqui com seu contato.
        self.chat_mode = {}  
        self.setup_llm()
        self.setup_custom_prompts()
        self.load_index()
        self.setup_handlers()
    
    def setup_llm(self):
        """Configuração do LLM e modelo de embeddings"""
        Settings.llm = Ollama(model="llama3.2", request_timeout=120.0)
        Settings.embed_model = OllamaEmbedding(
            model_name="nomic-embed-text",
            base_url="http://localhost:11434"
        )
        Settings.num_output = 256
        Settings.context_window = 4096
    
    def setup_custom_prompts(self):
        """Configura os prompts personalizados"""
        qa_prompt_str = (
            "As informações de contexto estão abaixo.\n"
            "Seu nome é uma Assistente de vendas.\n"
            "Está oferecendo um carro usado e não novo. \n"
            "Seja simpático.\n"
            "Responda sempre em português.\n"
            "Haja como se você estivesse vendendo o Siena 1.4, ano 2010, baseado na base de dados fornecida.\n"
            "No final de cada resposta, lembre o cliente que para negociações reais, utilize o comando /help para obter o contato do vendedor.\n"
            "{context_str}\n"
            "---------------------\n"
            "Dadas as informações de contexto, responda à pergunta de forma breve: {query_str}\n "
            "Se o usuário estiver perguntando sobre imagens ou fotos, sugira usar o comando /Fotos.\n"
        )
        
        self.qa_prompt = PromptTemplate(qa_prompt_str)

        refine_template_str = (
        "responda a {query_str} com uma resposta breve"
        " Seja fiel a sua base de dados.r"
        " NÃO INVENTE VALORES OU PROPOSTA"
        " NÃO FECHE NEGOCIO DIRECIONE AO /help PARA FECHAR E MARCAR VISITA"
        )
        self.refine_template = PromptTemplate(refine_template_str)
        
#----------------------------------------------------------------------------------    
    
    def load_index(self):
        """Carrega ou recria o índice de documentos"""
        try:
            data = SimpleDirectoryReader(input_dir=self.data_dir).load_data()
            self.index = VectorStoreIndex.from_documents(data)
            self.query_engine = self.index.as_query_engine(
                text_qa_template=self.qa_prompt,refine_template=self.refine_template
            )
            print("✅ Base de dados carregada com sucesso!")
        except Exception as e:
            print(f"❌ Erro ao carregar índice: {e}")
            self.index = None
            self.query_engine = None

#------------------------------------------------------------------------
    
    def setup_handlers(self):
        """Configura todos os handlers do bot"""
        
        @self.bot.message_handler(commands=['start'])
        def start(message):
            welcome_text = """
                🚗 Bem-vindo ao Assistente Virtual de Vendas!

                Sou uma IA especializada em fornecer informações sobre o Siena 1.4 2010.
                
                Comandos disponíveis:
                /help - Contato do vendedor via WhatsApp
                /Fotos - Visualizar fotos do veículo
                /chat - Conversar com a IA sobre o veículo
                /menu - Voltar ao menu principal
                
                ⚠️ Importante: Sou apenas informativo. Para negociações e visitas, use /help para contatar o vendedor.
                
                Como posso ajudar você hoje?
            
            """
            self.chat_mode[message.chat.id] = False  # Desativa modo chat
            self.bot.reply_to(message, welcome_text)

        @self.bot.message_handler(commands=['help'])
        def help(message):
            help_text = f"""
📱 <b>Contato do Vendedor</b>

Para negociações, visitas ou mais informações, entre em contato via WhatsApp:

wa.me/{self.whatsapp_number}

Horário de atendimento: 8h às 18h
            """
            
            self.bot.reply_to(message, help_text)

        @self.bot.message_handler(commands=['menu'])
        def menu(message):
            self.chat_mode[message.chat.id] = False  # Desativa modo chat
            menu_text = """
🔄 Menu Principal

Escolha uma opção:
/help - Contato do vendedor
/Fotos - Ver fotos do veículo
/chat - Conversar com a IA

⚠️ Lembre-se: Sou apenas informativo.
            """
            self.bot.reply_to(message, menu_text)

        @self.bot.message_handler(commands=['chat'])
        def start_chat(message):
            if not self.query_engine:
                self.bot.reply_to(message, 
                    "❌ Sistema não inicializado. Aguarde um momento e tente novamente.")
                self.load_index()  # Tenta recarregar o índice
                return
                
            self.chat_mode[message.chat.id] = True  # Ativa modo chat
            chat_intro = """
💬 Modo Conversa Ativado!

Agora você pode me fazer perguntas sobre o Siena 1.4 2010.
Responderei suas dúvidas baseado nas informações disponíveis sobre:
- Características do veículo
- Estado de conservação estética
- Estado de conservação mecânica
- Documentação
- Proposta

Use /menu para voltar ao menu principal.

Como posso ajudar?
            """
            self.bot.reply_to(message, chat_intro)

#----------------------------------------------------------------------
        
        @self.bot.message_handler(commands=['Fotos'])
        def gallery(message):
            try:
                photo_dir = "./imagens_estética"  # Diretório com as fotos
                photos = [f for f in os.listdir(photo_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                
                if not photos:
                    self.bot.reply_to(message, "Desculpe, não há fotos disponíveis no momento.")
                    return
                
                # Envia fotos individualmente
                for photo in photos[:10]:  # Limita a 10 fotos
                    try:
                        with open(os.path.join(photo_dir, photo), 'rb') as photo_file:
                            self.bot.send_photo(
                                message.chat.id,
                                photo_file,
                                caption=f"Foto: {photo}"
                            )
                    except Exception as e:
                        print(f"Erro ao enviar foto {photo}: {str(e)}")
                        continue
                
                self.bot.reply_to(message, "✅ Fotos enviadas! Para negociar, use /help para obter o contato do vendedor.")
                
            except Exception as e:
                self.bot.reply_to(message, f"Erro ao carregar as fotos: {str(e)}")


#----------------------------------------------------------------------

        
        @self.bot.message_handler(func=lambda message: True)
        def handle_query(message):
            if message.text.startswith('/'):
                return
                
            chat_id = str(message.chat.id)
            
            # Verifica se está no modo chat
            if message.chat.id not in self.chat_mode or not self.chat_mode[message.chat.id]:
                response_text = (
                    "Olá! Para iniciar uma conversa comigo sobre o Siena 1.4 2010, use o comando /chat\n\n"
                    "Outros comandos disponíveis:\n"
                    "/Fotos - Ver fotos do veículo\n"
                    "/help - Contato do vendedor via WhatsApp"
                )
                self.bot.reply_to(message, response_text)
                return
            
            if not self.query_engine:
                self.bot.reply_to(message, 
                    "❌ Sistema não inicializado. Aguarde um momento e tente novamente.")
                self.load_index()  # Tenta recarregar o índice
                return
            
            try:
                processing_msg = self.bot.reply_to(message, "🤔 Processando sua pergunta...")
                
                # Obtém resposta do LLM
                response = self.query_engine.query(message.text)
                response_text = str(response)
                
                # Atualiza histórico
                if chat_id not in self.conversation_history:
                    self.conversation_history[chat_id] = []
                    
                self.conversation_history[chat_id].append({
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'question': message.text,
                    'answer': response_text
                })
                
                # Envia resposta
                self.bot.edit_message_text(
                    chat_id=message.chat.id,
                    message_id=processing_msg.message_id,
                    text=response_text,
                    parse_mode='HTML'
                )
                
            except Exception as e:
                error_msg = f"❌ Erro ao processar pergunta: {str(e)}"
                self.bot.edit_message_text(
                    chat_id=message.chat.id,
                    message_id=processing_msg.message_id,
                    text=error_msg
                )
#----------------------------------------------------------------------

    
    def run(self):
        """Inicia o bot"""
        print("🤖 Bot iniciado...")
        self.bot.infinity_polling()

if __name__ == "__main__":
    BOT_TOKEN = bot_key #crie uma varieavel e adicione sua chave do telegram
    bot = EnhancedTelegramBot(BOT_TOKEN)
    bot.run()